In [37]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score

import nltk
from nltk import *
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import sys
import codecs
import string
import random
import re

import requests
import urllib
from bs4 import BeautifulSoup
from readability.readability import Document # https://github.com/buriy/python-readability. Tried Goose, Newspaper (python libraries on Github). Bad results.
from http.cookiejar import CookieJar # 

# Data Exploration
First, load in science article news data, which contains basic information of science articles including URL and description. I previously mined the articles from Stat News, Science Daily, and Scientific American, but they are missing a lot of crucial information for the goal of this text project.

In [10]:
news_df = pd.read_json("data/news.json")
print('Shape of data (rows, columns):', news_df.shape, '\n')
print('Column Names: ', news_df.columns) # flagged means whether or not the article is incomplete or inaccurate

Shape of data (rows, columns): (4000, 12) 

Column Names:  Index(['articleUrl', 'author', 'categories', 'createdAt', 'date',
       'description', 'flagged', 'imageUrl', 'slug', 'source', 'sourceUrl',
       'title'],
      dtype='object')


I have a couple of <b>goals</b> in mind for this text data: exploring some interesting features about the articles and extracting keywords. Although there is also a "categories" column in this data, it would be nice to create a model to categorize into categories -- for example, "Physics" or "Artificial Intelligence" -- to be able to compare these articles perhaps with new ones without the "categories" label. Also, it would be cool to see if article writers sometimes "incorrectly" categorizes an article.<br><br>
To clean up this data a bit, let's also remove a few unnecesary columns. For example, since "description" is unclean and can be replaced by the full "article, this column can be deleted.

In [11]:
# new article features to be populated
news_df['article'] = '' # full article
news_df['num_words_not_in_dictionary'] = 0 # number of words not in English dictionary (super scientific words)
news_df['stopwords2words'] = 0 # ratio of stopwords to total words
news_df['readability_score'] = 0 # readability score probably based on Flesch–Kincaid
news_df['sentiment_polarity'] = 0 # the sentiment polarity within an article (range of emotions)
news_df = news_df.drop(['author', 'createdAt', 'imageUrl', 'slug', 'date', 'description'], axis=1)
news_df = news_df.replace(np.nan, 0) # replace NaN with 0 for "flagged" column
news_df.head()

articleUrl        categories  \
0  https://www.statnews.com/2017/02/09/dana-farbe...  [science policy]   
1  https://www.statnews.com/2017/02/09/antibiotic...  [science policy]   
2  https://www.statnews.com/2017/02/07/scientist-...  [science policy]   
3  https://www.statnews.com/2017/02/09/twitter-ho...  [science policy]   
4  https://www.statnews.com/2017/02/09/obamacare-...  [science policy]   

   flagged     source                 sourceUrl  \
0      1.0  Stat News  https://www.statnews.com   
1      1.0  Stat News  https://www.statnews.com   
2      0.0  Stat News  https://www.statnews.com   
3      0.0  Stat News  https://www.statnews.com   
4      0.0  Stat News  https://www.statnews.com   

                                               title article  \
0  Dana-Farber will avoid ‘controversial venues’ ...           
1  Why your doctor’s advice to take all your anti...           
2  A lot of Americans don’t know a single scienti...           
3  The doctor is in: 13 clinicians to follow on T...           
4  Patient groups fear Obamacare repeal could und...           

   num_words_not_in_dictionary  stopwords2words  readability_score  \
0                            0                0                  0   
1                            0                0                  0   
2                            0                0                  0   
3                            0                0                  0   
4                            0                0                  0   

   sentiment_polarity  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0

# Scraping the Full Article Texts
First, populate the "article" column bye the whole article from the URL (will clean those full texts up later).<br> <b>NOTE:</b> The following cell will take a long time, so it is commented. The cell after it will directly load in the result of the cell.

In [8]:
# for i in range(len(news_df['articleUrl'])):
#     try:
#         url = news_df['articleUrl'][i] # Bad: 2841, 3013, 3015, 3077, 3091, 3092
#         opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor)
#         opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/535.7 (KHTML, like Gecko) Chrome/16.0.912.77 Safari/535.7')]
#         html =  opener.open(url).read().decode('utf-8')
#         readable_article = Document(html).summary()
#         soup = BeautifulSoup(readable_article, "lxml")
#         text = soup.get_text()
#         news_df.loc[i, 'article'] = text
#         print(i, " ", end="")
#         break
#     except:
#         print(str(i) + "error", " ", end="")

In [78]:
# Run this to skip above steps
news_df = pd.read_json("data/news_with_articles.json")
news_df = news_df.sort_index(ascending=True)
news_df = news_df[['articleUrl', 'categories', 'flagged', 'source', 'sourceUrl', 'title', 'article', 
             'num_words_not_in_dictionary', 'stopwords2words', 'readability_score', 'sentiment_polarity']]
news_df.head()

articleUrl        categories  \
0  https://www.statnews.com/2017/02/09/dana-farbe...  [science policy]   
1  https://www.statnews.com/2017/02/09/antibiotic...  [science policy]   
2  https://www.statnews.com/2017/02/07/scientist-...  [science policy]   
3  https://www.statnews.com/2017/02/09/twitter-ho...  [science policy]   
4  https://www.statnews.com/2017/02/09/obamacare-...  [science policy]   

   flagged     source                 sourceUrl  \
0        1  Stat News  https://www.statnews.com   
1        1  Stat News  https://www.statnews.com   
2        0  Stat News  https://www.statnews.com   
3        0  Stat News  https://www.statnews.com   
4        0  Stat News  https://www.statnews.com   

                                               title  \
0  Dana-Farber will avoid ‘controversial venues’ ...   
1  Why your doctor’s advice to take all your anti...   
2  A lot of Americans don’t know a single scienti...   
3  The doctor is in: 13 clinicians to follow on T...   
4  Patient groups fear Obamacare repeal could und...   

                                             article  \
0  \nT\nhe Dana-Farber Cancer Institute, which ha...   
1  \nY\nou’ve heard it many times before from you...   
2  \nI\n’ve recently been thinking about this: Th...   
3  \nC\nlinical medicine is rapidly changing in t...   
4  \nW\nASHINGTON — As Republicans confront the t...   

   num_words_not_in_dictionary  stopwords2words  readability_score  \
0                            0                0                  0   
1                            0                0                  0   
2                            0                0                  0   
3                            0                0                  0   
4                            0                0                  0   

   sentiment_polarity  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0

Web scraping is very messy, so we have to clean the "article" column. We will keep the '\n' characters to be able to split the text into paragraphs (without the use of NLTK).<br>
On closer ispection of the articles from "Science Daily," many articles did not scrape correctly. I will remove them now. This is the downsides of webscraping: so much noise in data.<br>
Let's also save the list of articles to use as a corpus.

In [80]:
news_df = news_df.replace({'\xa0': ' '}, regex=True) # clean these empty null characters.
news_df['article'][19]

'\n \n\n'

In [81]:
for i in range(len(news_df)):
    if len(news_df['article'][i]) < 10:
        news_df = news_df.drop(i)
news_df = news_df.reset_index(drop=True)
news_df

articleUrl  \
0     https://www.statnews.com/2017/02/09/dana-farbe...   
1     https://www.statnews.com/2017/02/09/antibiotic...   
2     https://www.statnews.com/2017/02/07/scientist-...   
3     https://www.statnews.com/2017/02/09/twitter-ho...   
4     https://www.statnews.com/2017/02/09/obamacare-...   
5     https://www.statnews.com/2017/02/08/drug-names...   
6     https://www.statnews.com/2017/02/09/alzheimers...   
7     https://www.statnews.com/2016/03/28/google-lif...   
8     https://www.statnews.com/2017/02/06/mental-hea...   
9     https://www.statnews.com/2017/02/09/doctors-tr...   
10    https://www.statnews.com/2017/02/09/injury-sta...   
11    https://www.statnews.com/2017/02/08/social-med...   
12    https://www.statnews.com/2017/02/09/harvard-re...   
13    https://www.statnews.com/2017/02/06/health-ins...   
14    https://www.statnews.com/2017/02/09/margaret-c...   
15    https://www.statnews.com/2017/02/07/trump-iraq...   
16    https://www.statnews.com/2017/02/08/toledo-alz...   
17    https://www.statnews.com/2017/01/30/trump-ment...   
18    https://www.statnews.com/2017/01/30/fda-regula...   
19    http://neurosciencenews.com/balloon-hearing-te...   
20    https://www.statnews.com/2017/02/06/trump-biop...   
21    https://www.statnews.com/pharmalot/2017/02/08/...   
22    https://www.statnews.com/pharmalot/2017/02/07/...   
23    https://www.statnews.com/pharmalot/2017/02/06/...   
24    https://www.statnews.com/pharmalot/2017/02/03/...   
25    https://www.statnews.com/pharmalot/2017/02/03/...   
26    https://www.statnews.com/pharmalot/2017/02/02/...   
27    https://www.statnews.com/pharmalot/2017/02/09/...   
28    https://www.statnews.com/pharmalot/2017/02/08/...   
29    https://www.statnews.com/pharmalot/2017/02/06/...   
...                                                 ...   
2120  https://www.scientificamerican.com/article/muo...   
2121  https://www.scientificamerican.com/slideshow/b...   
2122  https://www.scientificamerican.com/podcast/epi...   
2123  https://www.scientificamerican.com/article/the...   
2124  https://www.scientificamerican.com/article/det...   
2125  https://www.scientificamerican.com/article/int...   
2126  https://www.scientificamerican.com/article/an-...   
2127  https://www.scientificamerican.com/article/the...   
2128  https://www.scientificamerican.com/article/nas...   
2129  https://www.scientificamerican.com/article/tru...   
2130  https://www.scientificamerican.com/article/tru...   
2131  https://www.scientificamerican.com/video/scien...   
2132  https://www.scientificamerican.com/article/the...   
2133  https://www.scientificamerican.com/article/cli...   
2134  https://www.scientificamerican.com/article/the...   
2135  https://www.scientificamerican.com/article/5-i...   
2136  https://www.scientificamerican.com/article/tru...   
2137  https://www.scientificamerican.com/article/we-...   
2138  https://www.scientificamerican.com/article/one...   
2139  https://www.scientificamerican.com/article/u-s...   
2140  https://www.scientificamerican.com/article/tru...   
2141  https://www.scientificamerican.com/article/ear...   
2142  https://www.scientificamerican.com/article/one...   
2143  https://www.scientificamerican.com/article/the...   
2144  https://www.scientificamerican.com/article/the...   
2145  https://www.scientificamerican.com/article/tru...   
2146  https://www.scientificamerican.com/article/ink...   
2147  https://www.scientificamerican.com/article/ant...   
2148  https://www.scientificamerican.com/article/the...   
2149  https://www.scientificamerican.com/article/phy...   

                   categories  flagged               source  \
0            [science policy]        1            Stat News   
1            [science policy]        1            Stat News   
2            [science policy]        0            Stat News   
3            [science policy]        0            Stat News   
4            [science policy]        0            Stat News   
5       

In [82]:
articles = [str(s) for s in news_df['article']] # create a corpus of all articles
articles[0]

'\nT\nhe Dana-Farber Cancer Institute, which has faced strong criticism for its decision to hold a fundraiser this month at President Trump’s Mar-a-Lago Club, said Thursday that it would avoid “controversial venues” in the future.\nThe hospital’s chief executive, Dr. Laurie Glimcher, said in a statement that its upcoming event, scheduled for Feb. 18, had become a “lightning rod” for some.\n“In the future we will avoid controversial venues that may distract from our focus on cancer care and research,” Glimcher said, adding that this year’s event would go on.\nadvertisement\nThe announcement from the Boston hospital could increase pressure on other clinics and charities that have faced calls to cancel events at Mar-a-Lago.\nAmong them is the Cleveland Clinic, which had one of its staff doctors barred from entering the United States after President Trump’s executive order on immigration.\n\nDana-Farber has held a fundraiser at Mar-a-Lago every year since 2011, and once in 2008. It expects

### Stopwords, stemming, and tokenization to clean the text even more
tokenize_and_stem: tokenizes and also stems each token.<br>
tokenize_only: tokenizes the text only to be able to convert stemmed tokens back to the full word.<br>

In [256]:
# load nltk's English stopwords
stopwords = nltk.corpus.stopwords.words('english')
# load nltk's SnowballStemmer
stemmer = SnowballStemmer("english")

# Define a tokenizer and stemmer that returns the set of stems of the text
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if len(token) <= 1: # remove one character words, "a" apostraphe s.
            continue
        if token.isalpha():
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if len(token) <= 1: # remove one character words, "a" apostraphe s.
            continue
        if token.isalpha():
            filtered_tokens.append(token)
    return filtered_tokens

Use stemming + tokenizing and tokenizing only functions to create two vocabulary lists.<br>
These two lists create a Pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column (to convert stem into its first full word/token, could be many, but take first for simplicity).<br>
Stemming is not the best... especially when it comes to Proper Nouns and other special words.

In [257]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in articles:
    allwords_stemmed = tokenize_and_stem(i) #for each article in "articles", tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
vocab_df = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed) # map stem to first actual word for simplicity
print('Shape of vocab_df (rows, columns):', vocab_df.shape, '\n')
vocab_df.head()

Shape of vocab_df (rows, columns): (1662526, 1) 



words
he               he
cancer       cancer
institut  institute
which         which
has             has

### Also, let's start filling out the other columns since we have the full article texts now


In [87]:
stopwords = set(w.lower() for w in nltk.corpus.stopwords.words('english'))

def stopwords_to_words(s):
    raw = nltk.word_tokenize(s)
    text = nltk.Text(raw)
    text_vocab = set(w.lower() for w in text if w.isalpha())
    return len(text_vocab.intersection(stopwords)) / len(text_vocab)
    
for i in range(len(news_df['articleUrl'])):
    try:
        news_df.loc[i, 'stopwords2words'] = stopwords_to_words(news_df['article'][i])
        print(i, ':', news_df['articleUrl'][i], ":", stopwords_to_words(news_df['article'][i]))
    except:
        pass # keeps empty articles 0.0

0 : https://www.statnews.com/2017/02/09/dana-farber-mar-lago/ : 0.25
1 : https://www.statnews.com/2017/02/09/antibiotics-resistance-superbugs/ : 0.17032967032967034
2 : https://www.statnews.com/2017/02/07/scientist-march-trump/ : 0.20891364902506965
3 : https://www.statnews.com/2017/02/09/twitter-hospitals-medicine/ : 0.14606741573033707
4 : https://www.statnews.com/2017/02/09/obamacare-repeal-preexisting-conditions/ : 0.18032786885245902
5 : https://www.statnews.com/2017/02/08/drug-names-process/ : 0.11202830188679246
6 : https://www.statnews.com/2017/02/09/alzheimers-disease-statins/ : 0.18518518518518517
7 : https://www.statnews.com/2016/03/28/google-life-sciences-exodus/ : 0.08673894912427023
8 : https://www.statnews.com/2017/02/06/mental-health-college-students/ : 0.13237063778580024
9 : https://www.statnews.com/2017/02/09/doctors-training-social-change/ : 0.14678899082568808
10 : https://www.statnews.com/2017/02/09/injury-states-unusually-common/ : 0.22522522522522523
11 : https:

89 : https://www.scientificamerican.com/article/wetlands-can-help-fight-climate-change/ : 0.19230769230769232
90 : https://www.scientificamerican.com/article/march-for-science-set-for-earth-day/ : 0.296551724137931
91 : https://www.scientificamerican.com/article/science-and-the-u-s-supreme-court-the-cases-to-watch-in-2017/ : 0.1425339366515837
92 : https://www.scientificamerican.com/article/florida-has-seen-bad-effects-from-trump-like-climate-gag-orders/ : 0.16666666666666666
93 : https://www.scientificamerican.com/article/groundhog-day-is-hogwash/ : 0.4
94 : https://www.scientificamerican.com/article/beijing-vows-deep-cut-in-coal-use-in-2017-to-fight-smog/ : 0.18604651162790697
95 : https://www.scientificamerican.com/podcast/episode/the-arctics-anti-snowball-snowball-effect/ : 0.24285714285714285
96 : https://www.scientificamerican.com/article/e-u-looks-to-china-for-climate-leadership/ : 0.1640625
97 : https://www.scientificamerican.com/article/house-science-panel-to-hold-hearing-on-l

165 : http://www.the-scientist.com//?articles.view/articleNo/47611/title/Low-Social-Status-May-Weaken-Immune-System-in-Monkeys/ : 0.2556818181818182
166 : https://www.statnews.com/2017/01/25/vertex-pain-opioids/ : 0.2911392405063291
167 : https://www.statnews.com/2017/01/23/week-of-jan-23-what-to-watch-for-in-biopharma/ : 0.29850746268656714
168 : http://www.the-scientist.com//?articles.view/articleNo/48229/title/ACMG-Urges-Caution-When-Editing-Embryo-Genomes/ : 0.22297297297297297
169 : http://www.the-scientist.com//?articles.view/articleNo/47333/title/Immunity-in-the-Brain/ : 0.24096385542168675
170 : http://www.the-scientist.com//?articles.view/articleNo/47905/title/Pharma-Redo/ : 0.21428571428571427
171 : http://www.the-scientist.com//?articles.view/articleNo/48034/title/Abscisic-Acid-s-Role-in-Ferns-Finally-Determined/ : 0.1970074812967581
172 : http://www.the-scientist.com//?articles.view/articleNo/47993/title/How-Plant-Soil-Feedback-Affects-Ecological-Diversity/ : 0.196675900277

257 : http://neurosciencenews.com/brain-development-autism-6034/ : 0.14672686230248308
258 : http://neurosciencenews.com/decision-making-network-6050/ : 0.1566579634464752
259 : http://neurosciencenews.com/insomnia-sexual-function-6038/ : 0.21649484536082475
260 : https://www.statnews.com/2017/02/09/cancer-drug-benefit-price/ : 0.32941176470588235
261 : https://www.statnews.com/2017/01/12/nevada-woman-superbug-resistant/ : 0.18610421836228289
262 : https://www.statnews.com/pharmalot/2017/02/09/drug-prices-pharmacy-benefits-managers/ : 0.24175824175824176
263 : https://www.scientificamerican.com/article/republicans-offer-to-tax-carbon-emissions/ : 0.1731266149870801
264 : https://www.scientificamerican.com/article/house-science-committee-may-soon-try-to-weaken-the-epa/ : 0.1540041067761807
265 : https://www.scientificamerican.com/article/house-science-committee-may-soon-try-to-weaken-the-epa/ : 0.1540041067761807
266 : https://www.scientificamerican.com/article/republicans-offer-to-tax-

356 : https://www.statnews.com/2017/01/13/tom-price-innate-immunotherapeutics/ : 0.12055335968379446
357 : https://www.statnews.com/2017/02/12/ex-drug-company-ceo-shkreli-speak-harvard/ : 0.24468085106382978
358 : https://www.statnews.com/2017/01/18/anorexia-forced-treatment/ : 0.1649782923299566
359 : https://www.statnews.com/2016/03/17/medical-students-match-day/ : 0.181640625
360 : https://www.statnews.com/2016/03/31/dna-shape-double-helix-dekker/ : 0.225
361 : https://www.statnews.com/2017/01/26/chimera-humans-animals-ethics/ : 0.1787709497206704
362 : https://www.statnews.com/2016/07/06/epipen-prices-allergies/ : 0.16962524654832348
363 : https://www.statnews.com/2017/01/10/kennedy-vaccine-book/ : 0.12351543942992874
364 : https://www.statnews.com/2017/01/28/egg-donors-risks/ : 0.15384615384615385
365 : https://www.statnews.com/2017/02/13/california-hiv-transmission-criminal-penalties/ : 0.2026315789473684
366 : https://www.statnews.com/2017/02/13/flumist-mystery/ : 0.329787234042

...

1581 : https://www.statnews.com/pharmalot/2017/03/20/amgen-cholesterol-drug-prices/ : 0.22962962962962963
1582 : https://www.bostonglobe.com/business/2017/03/19/biopharma-facing-stiffer-opposition/EmzTSsLhlanZ33E6AOUteO/story.html : 0.18509615384615385
1583 : http://www.news-medical.net/news/20170320/Linking-RNA-to-autoimmune-diseases.aspx : 0.15917602996254682
1584 : https://www.statnews.com/2017/03/20/esperion-amgen-pcsk9/ : 0.3053435114503817
1585 : https://www.statnews.com/2017/03/20/hospitals-rural-health-care-aca-trump/ : 0.20316027088036118
1586 : https://www.youtube.com/watch?v=AVwUkBAwjgs&list=LLPEk13IV9vkbxwtDR0KpWOA&index=8 : 0.5
1587 : https://kynplex.com : 0.20512820512820512
1588 : https://kynplex.com : 0.20512820512820512
1589 : https://kynplex.com : 0.20512820512820512
1590 : https://kynplex.com : 0.20512820512820512
1591 : https://kynplex.com : 0.20512820512820512
1592 : https://www.statnews.com/2016/12/20/transgender-youth-doctors-clinics/ : 0.125
1593 : https://www.s

1687 : http://neurosciencenews.com/sensory-signals-neuroscience-6278/ : 0.14741035856573706
1688 : http://neurosciencenews.com/parkinsons-exercise-neurology-6282/ : 0.1597444089456869
1689 : http://neurosciencenews.com/immune-system-vaccines-6287/ : 0.16389548693586697
1690 : http://neurosciencenews.com/parkinsons-exercise-neurology-6282/ : 0.1597444089456869
1691 : http://neurosciencenews.com/pleasure-amygdala-neuroscience-6286/ : 0.17506631299734748
1692 : http://neurosciencenews.com/brain-rewiring-senses-blind-6276/ : 0.1455026455026455
1693 : https://www.scientificamerican.com/article/california-adopts-strict-rules-for-methane-emissions/ : 0.20233463035019456
1694 : https://www.scientificamerican.com/article/ethical-guidelines-on-lab-grown-embryos-beg-for-revamping-scientists-say/ : 0.15514018691588785
1695 : http://neurosciencenews.com/pleasure-amygdala-neuroscience-6286/ : 0.17506631299734748
1696 : http://www.news-medical.net/news/20170323/Analysis-of-antibodies-could-be-new-too

1790 : http://www.the-scientist.com//?articles.view/articleNo/48973/title/Making-CAR-T-Cell-Therapy-Safer/ : 0.11833550065019506
1791 : http://www.the-scientist.com//?articles.view/articleNo/48903/title/Starvation-Response-Triggers-Melanoma-Invasion/ : 0.19014084507042253
1792 : http://www.the-scientist.com//?articles.view/articleNo/49047/title/Cancer-Genomes/ : 0.20833333333333334
1793 : http://www.the-scientist.com//?articles.view/articleNo/48909/title/Location--Location--Location/ : 0.12278630460448642
1794 : http://www.the-scientist.com//?articles.view/articleNo/49000/title/Neoantigens-Enable-Personalized-Cancer-Immunotherapy/ : 0.09871794871794871
1795 : http://www.the-scientist.com//?articles.view/articleNo/48973/title/Making-CAR-T-Cell-Therapy-Safer/ : 0.11833550065019506
1796 : http://www.the-scientist.com//?articles.view/articleNo/49003/title/Circadian-Rhythms-Influence-Treatment-Effects/ : 0.09655172413793103
1797 : http://www.the-scientist.com//?articles.view/articleNo/48910

1897 : http://news.mit.edu/2017/csail-launches-artificial-intelligence-initiative-with-industry-0406 : 0.15384615384615385
1898 : https://www.scientificamerican.com/article/politicians-shouldn-rsquo-t-troll-through-scientists-rsquo-e-mails/ : 0.20930232558139536
1899 : https://www.scientificamerican.com/podcast/episode/extreme-storms-are-extreme-eroders/ : 0.24380165289256198
1900 : https://www.scientificamerican.com/article/earths-co2-could-spike-to-a-level-not-seen-since-the-dinosaurs/ : 0.22012578616352202
1901 : https://www.scientificamerican.com/article/a-tiny-detection-chip-could-find-methane-leaks-autonomously/ : 0.1590909090909091
1902 : https://www.scientificamerican.com/article/most-americans-oppose-climate-science-cuts/ : 0.24630541871921183
1903 : https://www.scientificamerican.com/article/most-americans-oppose-climate-science-cuts/ : 0.24630541871921183
1904 : https://www.scientificamerican.com/article/astronomers-observe-milky-way-like-galaxies-in-early-universe/ : 0.1695

2010 : http://neurosciencenews.com/alzheimers-music-therapy-6414/ : 0.2247557003257329
2011 : http://neurosciencenews.com/stem-cell-brain-tissue-6407/ : 0.16159250585480095
2012 : http://neurosciencenews.com/chemobrain-cancer-6397/ : 0.15085158150851583
2013 : http://neurosciencenews.com/artificial-intelligence-human-prejudice-6411/ : 0.12382739212007504
2014 : http://neurosciencenews.com/immunotherapy-glioblastoma-6416/ : 0.11555555555555555
2015 : http://neurosciencenews.com/alcohol-dependence-amygdala-6399/ : 0.132013201320132
2016 : http://neurosciencenews.com/artificial-intelligence-human-prejudice-6411/ : 0.12382739212007504
2017 : http://neurosciencenews.com/oxytocin-ptsd-addiction-6412/ : 0.2197452229299363
2018 : http://neurosciencenews.com/cognitive-map-scaling-6403/ : 0.13852813852813853
2019 : http://neurosciencenews.com/sensory-cell-regeneration-hearing-6391/ : 0.14772727272727273
2020 : http://neurosciencenews.com/oxytocin-ptsd-addiction-6412/ : 0.2197452229299363
2021 : 

In [92]:
# WARNING: Takes a long time. Might not be accurate but relatively should determine which article has more complex words.

# def num_unusual_words(s):
#     raw = nltk.word_tokenize(s)
#     text = nltk.Text(raw)
#     text_vocab = set(w.lower() for w in text if w.isalpha())
#     english_vocab = set(w.lower() for w in nltk.corpus.words.words())
#     unusual = text_vocab - english_vocab
#     return len(sorted(unusual))

# for i in range(len(news_df['articleUrl'])):
#     try:
#         news_df.loc[i, 'num_words_not_in_dictionary'] = num_unusual_words(news_df['article'][i])
#         print(i, ':', news_df['articleUrl'][i], ":", num_unusual_words(news_df['article'][i]))
#     except:
#         pass # keeps empty articles 0.0

0 : https://www.statnews.com/2017/02/09/dana-farber-mar-lago/ : 34
1 : https://www.statnews.com/2017/02/09/antibiotics-resistance-superbugs/ : 116
2 : https://www.statnews.com/2017/02/07/scientist-march-trump/ : 76
3 : https://www.statnews.com/2017/02/09/twitter-hospitals-medicine/ : 105
4 : https://www.statnews.com/2017/02/09/obamacare-repeal-preexisting-conditions/ : 84
5 : https://www.statnews.com/2017/02/08/drug-names-process/ : 231
6 : https://www.statnews.com/2017/02/09/alzheimers-disease-statins/ : 85
7 : https://www.statnews.com/2016/03/28/google-life-sciences-exodus/ : 311
8 : https://www.statnews.com/2017/02/06/mental-health-college-students/ : 192
9 : https://www.statnews.com/2017/02/09/doctors-training-social-change/ : 102
10 : https://www.statnews.com/2017/02/09/injury-states-unusually-common/ : 27
11 : https://www.statnews.com/2017/02/08/social-media-compulsion-mental-illness/ : 130
12 : https://www.statnews.com/2017/02/09/harvard-reunion-sugar-study/ : 72
13 : https://ww

89 : https://www.scientificamerican.com/article/wetlands-can-help-fight-climate-change/ : 63
90 : https://www.scientificamerican.com/article/march-for-science-set-for-earth-day/ : 26
91 : https://www.scientificamerican.com/article/science-and-the-u-s-supreme-court-the-cases-to-watch-in-2017/ : 107
92 : https://www.scientificamerican.com/article/florida-has-seen-bad-effects-from-trump-like-climate-gag-orders/ : 102
93 : https://www.scientificamerican.com/article/groundhog-day-is-hogwash/ : 4
94 : https://www.scientificamerican.com/article/beijing-vows-deep-cut-in-coal-use-in-2017-to-fight-smog/ : 39
95 : https://www.scientificamerican.com/podcast/episode/the-arctics-anti-snowball-snowball-effect/ : 39
96 : https://www.scientificamerican.com/article/e-u-looks-to-china-for-climate-leadership/ : 74
97 : https://www.scientificamerican.com/article/house-science-panel-to-hold-hearing-on-ldquo-making-the-epa-great-again-rdquo/ : 37
98 : https://www.scientificamerican.com/article/house-science-

169 : http://www.the-scientist.com//?articles.view/articleNo/47333/title/Immunity-in-the-Brain/ : 36
170 : http://www.the-scientist.com//?articles.view/articleNo/47905/title/Pharma-Redo/ : 8
171 : http://www.the-scientist.com//?articles.view/articleNo/48034/title/Abscisic-Acid-s-Role-in-Ferns-Finally-Determined/ : 100
172 : http://www.the-scientist.com//?articles.view/articleNo/47993/title/How-Plant-Soil-Feedback-Affects-Ecological-Diversity/ : 77
173 : http://www.the-scientist.com//?articles.view/articleNo/48376/title/Toward-Killing-Cancer-with-Bacteria/ : 77
174 : https://www.statnews.com/2017/02/03/refugees-detroit-stress/ : 120
175 : http://www.bostonglobe.com/metro/2017/02/04/dressed-scrubs-she-roamed-hospital-but-she-wasn-supposed-there/3OkuPYs4PklE3MGdeLirhM/story.html : 134
176 : https://www.statnews.com/2017/02/09/cuba-doctors-us/ : 82
177 : https://www.statnews.com/2017/02/03/super-bowl-can-make-you-sick/ : 78
178 : https://www.statnews.com/2017/02/06/doctors-work-life-balanc

273 : https://www.statnews.com/2017/02/07/let-patients-talk/ : 95
274 : https://www.statnews.com/2016/02/11/stat-harvard-poll-gene-editing/ : 92
275 : https://www.statnews.com/2017/02/09/malaria-odor-mosquitoes/ : 67
276 : https://www.statnews.com/2016/09/01/optogenetics/ : 278
277 : https://www.statnews.com/pharmalot/2017/02/10/trump-fda-drug-prices-cancer/ : 15
278 : https://www.statnews.com/2017/02/10/pharma-acquisitions/ : 19
279 : https://www.statnews.com/2017/02/10/pharma-tax-break-jobs/ : 16
280 : https://www.statnews.com/2017/02/10/syria-refugees-boston-childrens/ : 37
281 : https://www.scientificamerican.com/article/arctic-2-0-what-happens-after-all-the-ice-goes/ : 210
282 : https://www.statnews.com/2017/02/10/price-hhs-confirmation/ : 100
283 : https://www.statnews.com/2017/02/10/medicaid-tinkering-trump-verma/ : 82
284 : https://www.statnews.com/2017/02/10/marathons-move-on-rare-disease-drug-martin-shrkeli-all-over-again/ : 16
285 : https://www.statnews.com/pharmalot/2017/02

378 : https://www.statnews.com/pharmalot/2017/02/13/sec-pharma-accounting-allergan/ : 21
379 : http://www.the-scientist.com//?articles.view/articleNo/47973/title/First-Bumblebee-Species-Declared-Endangered-in-U-S-/ : 40
380 : http://neurosciencenews.com/cognition-older-mothers-6107/ : 68
381 : http://neurosciencenews.com/big-brother-brain-scan-6105/ : 95
382 : http://neurosciencenews.com/cognition-older-mothers-6107/ : 68
383 : http://neurosciencenews.com/big-brother-brain-scan-6105/ : 95
384 : http://neurosciencenews.com/ptsd-genetics-memory-6107/ : 82
385 : http://neurosciencenews.com/big-brother-brain-scan-6105/ : 95
386 : http://neurosciencenews.com/ptsd-genetics-memory-6107/ : 82
387 : http://neurosciencenews.com/neurons-cancer-growth-6106/ : 98
388 : http://neurosciencenews.com/neurons-cancer-growth-6106/ : 98
389 : http://neurosciencenews.com/ptsd-genetics-memory-6107/ : 82
390 : http://neurosciencenews.com/neurons-cancer-growth-6106/ : 98
391 : http://neurosciencenews.com/big-b

...

1739 : https://www.statnews.com/2017/03/27/soon-shiong-cancer-video-questions/ : 188
1740 : http://www.news-medical.net/news/20170324/Researchers-develop-new-lab-on-a-chip-platform-that-aims-to-improve-pathogen-detection.aspx : 34
1741 : http://www.news-medical.net/news/20170329/Study-explores-new-immunotherapy-combinations-to-treat-prostate-cancer.aspx : 93
1742 : https://www.scientificamerican.com/article/hospitals-halt-hiring-projects-amid-uncertain-fate-of-obamacare/ : 99
1743 : https://www.scientificamerican.com/podcast/episode/whats-driving-the-self-driving-cars-rush/ : 3
1744 : https://www.statnews.com/2017/03/29/manchin-mylan-drug-pricing/ : 13
1745 : http://www.news-medical.net/news/20170325/UVA-researchers-discover-unexpected-interaction-between-mens-testes-and-immune-system.aspx : 52
1746 : https://www.statnews.com/2017/03/29/waivers-health-care-republicans-medicaid/ : 114
1747 : https://www.statnews.com/2017/03/29/mumps-outbreaks/ : 163
1748 : http://www.the-scientist.com//

1820 : https://www.statnews.com/2017/03/30/drug-prices-new-approved/ : 28
1821 : https://www.statnews.com/pharmalot/2017/03/30/glaxo-trump-fda-price/ : 23
1822 : https://www.statnews.com/pharmalot/2017/03/29/lawmakers-bill-drug-prices/ : 19
1823 : https://www.statnews.com/pharmalot/2017/03/29/uk-doctors-drug-makers/ : 15
1824 : https://www.statnews.com/pharmalot/2017/03/29/arizona-law-off-label-free-speech/ : 18
1825 : https://www.statnews.com/pharmalot/2017/03/29/fda-gottlieb-glaxo-mylan/ : 23
1826 : https://www.statnews.com/pharmalot/2017/03/28/mylan-epipen-medicaid-taxpayers/ : 16
1827 : http://www.the-scientist.com//?articles.view/articleNo/49000/title/Neoantigens-Enable-Personalized-Cancer-Immunotherapy/ : 212
1828 : http://www.the-scientist.com//?articles.view/articleNo/49005/title/Infographic--Targeting-Cancer-Antigens/ : 37
1829 : https://www.statnews.com/2017/03/31/moonshot-medicine-cancer-aging/ : 64
1830 : https://www.statnews.com/2017/04/04/preclinical-cellular-therapy-ovar

1905 : https://www.scientificamerican.com/article/epa-proposal-cuts-hundreds-of-climate-change-employees/ : 68
1906 : https://www.scientificamerican.com/article/epa-proposal-cuts-hundreds-of-climate-change-employees/ : 68
1907 : https://www.scientificamerican.com/article/states-challenge-trump-over-clean-power-plan/ : 41
1908 : https://www.scientificamerican.com/report/chemical-and-biological-weapons/ : 70
1909 : http://www.the-scientist.com//?articles.view/articleNo/49151/title/Viral-Trigger-for-Celiac-Disease-/ : 85
1910 : https://www.statnews.com/2016/05/06/science-janet-jacksons-pregnancy/ : 61
1911 : https://www.statnews.com/2017/04/06/crispr-pig-organs-transplant-luhan-yang/ : 259
1912 : https://www.statnews.com/2016/02/23/zika-olympics-first-opinion/ : 44
1913 : http://neurosciencenews.com/deep-sleep-aging-6349/ : 92
1914 : http://neurosciencenews.com/brain-navigation-6364/ : 109
1915 : http://neurosciencenews.com/ptsd-doxycycline-6365/ : 104
1916 : http://neurosciencenews.com/p

1984 : https://www.statnews.com/pharmalot/2017/04/05/fda-drugs-ema-approvals/ : 24
1985 : https://www.statnews.com/2017/04/11/where-do-former-officials-go-after-fda-look-down-k-street/ : 13
1986 : https://www.statnews.com/pharmalot/2017/04/10/stada-bain-cinven-teva-novo-diabetes/ : 29
1987 : https://www.statnews.com/pharmalot/2017/04/06/mallinckrodt-phrma-drug-prices/ : 21
1988 : https://www.statnews.com/pharmalot/2017/04/07/actelion-kfc-antibiotics-merck/ : 23
1989 : https://www.statnews.com/pharmalot/2017/04/05/democrats-trump-patents-drug-prices/ : 16
1990 : https://www.statnews.com/pharmalot/2017/04/07/winners-losers-gottlieb-fda-mallinckrodt/ : 32
1991 : https://www.statnews.com/2017/04/06/drug-development-price/ : 12
1992 : https://www.statnews.com/pharmalot/2017/04/05/cancer-doctors-right-to-try-laws/ : 9
1993 : https://www.statnews.com/2017/04/11/mylan-fda-warning-letter-nashik/ : 21
1994 : https://www.statnews.com/pharmalot/2017/04/06/gottlieb-fda-allergan-botox/ : 24
1995 : h

2080 : https://www.statnews.com/2017/04/27/gottlieb-help-vote/ : 13
2081 : https://www.statnews.com/2017/04/25/oncology-cancer-precision-medicine-gleevec/ : 123
2082 : https://www.statnews.com/2017/04/27/undergrads-drugs-tropical-diseases/ : 13
2083 : https://www.statnews.com/2017/04/24/mylan-west-virginia-lawsuit/ : 12
2084 : https://www.statnews.com/2016/05/02/adam-feuerstein-biotech/ : 336
2085 : https://www.statnews.com/2017/04/27/gottlieb-help-vote/ : 13
2086 : https://www.statnews.com/pharmalot/2017/04/27/novartis-korea-glaxo-fda/ : 25
2087 : https://www.statnews.com/pharmalot/2017/04/26/patent-cliff-biosimilars-generics/ : 20
2088 : https://www.statnews.com/pharmalot/2017/04/26/teva-collins-fda-mallinckrodt/ : 23
2089 : https://www.statnews.com/2017/04/25/gottlieb-fda-right-to-try/ : 20
2090 : https://www.statnews.com/pharmalot/2017/04/25/roche-india-biosimilar-competition/ : 17
2091 : https://www.statnews.com/pharmalot/2017/04/25/supreme-court-biosimilar-amgen-sandoz/ : 21
2092

"readability_score" is not super important for this project, but here are some interesting leads for it: https://datawarrior.wordpress.com/2016/03/29/flesch-kincaid-readability-measure/

In [96]:
# From: http://www.nltk.org/howto/sentiment.html
def sentiment_polarity(s):
    paragraphs = re.split('\n' , s)
    paragraphs = [p for p in paragraphs if p != '']
    sid = SentimentIntensityAnalyzer()
    neu = []
    compound = []
    for sentence in paragraphs:
        ss = sid.polarity_scores(sentence)
        neu.append(ss['neu'])
    return np.var(neu)

for i in range(len(news_df['articleUrl'])):
    try:
        news_df.loc[i, 'sentiment_polarity'] = sentiment_polarity(news_df['article'][i])
        print(i, ':', news_df['articleUrl'][i], ":", sentiment_polarity(news_df['article'][i]))
    except:
        pass # keeps empty articles 0.0

0 : https://www.statnews.com/2017/02/09/dana-farber-mar-lago/ : 0.0704219183673
1 : https://www.statnews.com/2017/02/09/antibiotics-resistance-superbugs/ : 0.0353557191358
2 : https://www.statnews.com/2017/02/07/scientist-march-trump/ : 0.0554019555556
3 : https://www.statnews.com/2017/02/09/twitter-hospitals-medicine/ : 0.04819336
4 : https://www.statnews.com/2017/02/09/obamacare-repeal-preexisting-conditions/ : 0.040396102071
5 : https://www.statnews.com/2017/02/08/drug-names-process/ : 0.0216973525377
6 : https://www.statnews.com/2017/02/09/alzheimers-disease-statins/ : 0.0433910495868
7 : https://www.statnews.com/2016/03/28/google-life-sciences-exodus/ : 0.0387522992879
8 : https://www.statnews.com/2017/02/06/mental-health-college-students/ : 0.0254864799899
9 : https://www.statnews.com/2017/02/09/doctors-training-social-change/ : 0.0498242376543
10 : https://www.statnews.com/2017/02/09/injury-states-unusually-common/ : 0.14703521
11 : https://www.statnews.com/2017/02/08/social-med

82 : https://www.scientificamerican.com/article/can-empirical-entertainment-rescue-science-in-an-alt-fact-world/ : 0.0106801540062
83 : https://www.scientificamerican.com/article/trump-supreme-court-nominee-would-put-agencies-on-short-leash/ : 0.00972099555556
84 : https://www.scientificamerican.com/article/newfound-source-of-mysterious-cosmic-bursts-poses-deeper-enigmas/ : 0.0218558875
85 : https://www.scientificamerican.com/podcast/episode/rapid-response-vaccines-for-epidemic-outbreaks/ : 0.22325625
86 : https://www.scientificamerican.com/article/did-a-changing-climate-wipe-out-the-giant-kangaroo/ : 0.00485616
87 : https://www.scientificamerican.com/article/the-epa-has-started-to-remove-obama-era-information/ : 0.00388298765432
88 : https://www.scientificamerican.com/article/a-tax-on-carbon-pollution-can-benefit-business/ : 0.008463234375
89 : https://www.scientificamerican.com/article/wetlands-can-help-fight-climate-change/ : 0.00480357333333
90 : https://www.scientificamerican.com/

159 : http://www.the-scientist.com//?articles.view/articleNo/48073/title/RNA-Interference-Between-Kingdoms/ : 0.21329758284
160 : http://www.the-scientist.com//?articles.view/articleNo/48391/title/Pardis-Sabeti--An-American-Scientist-Born-in-Iran/ : 0.190500336484
161 : http://www.the-scientist.com//?articles.view/articleNo/47853/title/Researchers-Call-for-Retraction-of-Paper-that-Questions-HPV-Vaccine/ : 0.167078429752
162 : http://www.the-scientist.com//?articles.view/articleNo/48329/title/Regulators-OK-Clinical-Trials-Using-Donor-Stem-Cells/ : 0.207444888889
163 : https://www.statnews.com/2017/01/18/crispr-cas3-locus-biosciences/ : 0.00442040816327
164 : http://www.the-scientist.com//?articles.view/articleNo/47973/title/First-Bumblebee-Species-Declared-Endangered-in-U-S-/ : 0.186359928889
165 : http://www.the-scientist.com//?articles.view/articleNo/47611/title/Low-Social-Status-May-Weaken-Immune-System-in-Monkeys/ : 0.189051
166 : https://www.statnews.com/2017/01/25/vertex-pain-opio

254 : http://neurosciencenews.com/sleep-deprivation-memory-6051/ : 0.0
255 : http://neurosciencenews.com/dbs-heroin-relapse-6039/ : 0.0
256 : http://neurosciencenews.com/social-attraction-neuroscience-6025/ : 0.0
257 : http://neurosciencenews.com/brain-development-autism-6034/ : 0.0
258 : http://neurosciencenews.com/decision-making-network-6050/ : 0.0
259 : http://neurosciencenews.com/insomnia-sexual-function-6038/ : 0.0
260 : https://www.statnews.com/2017/02/09/cancer-drug-benefit-price/ : 0.106826609375
261 : https://www.statnews.com/2017/01/12/nevada-woman-superbug-resistant/ : 0.0429879259259
262 : https://www.statnews.com/pharmalot/2017/02/09/drug-prices-pharmacy-benefits-managers/ : 0.106668
263 : https://www.scientificamerican.com/article/republicans-offer-to-tax-carbon-emissions/ : 0.00320576
264 : https://www.scientificamerican.com/article/house-science-committee-may-soon-try-to-weaken-the-epa/ : 0.00382467346939
265 : https://www.scientificamerican.com/article/house-science-c

350 : https://www.statnews.com/2017/01/03/aging-control-telomere-effect/ : 0.03813032
351 : https://www.statnews.com/2016/09/29/medical-students-learn-empathy/ : 0.0447882006173
352 : https://www.statnews.com/2016/01/05/detox-water-flush-fat/ : 0.0552285555556
353 : https://www.statnews.com/2016/11/08/zika-in-cuba/ : 0.0263529375372
354 : https://www.statnews.com/2016/01/21/crispr-conflicts-of-interest/ : 0.0467907482639
355 : https://www.statnews.com/2016/07/08/immune-therapies-juno-cancer/ : 0.0398118376691
356 : https://www.statnews.com/2017/01/13/tom-price-innate-immunotherapeutics/ : 0.0277520810249
357 : https://www.statnews.com/2017/02/12/ex-drug-company-ceo-shkreli-speak-harvard/ : 0.107741673469
358 : https://www.statnews.com/2017/01/18/anorexia-forced-treatment/ : 0.02879889
359 : https://www.statnews.com/2016/03/17/medical-students-match-day/ : 0.0276505789474
360 : https://www.statnews.com/2016/03/31/dna-shape-double-helix-dekker/ : 0.0738252430556
361 : https://www.statnew

...

1783 : http://www.the-scientist.com//?articles.view/articleNo/48912/title/Gel-Scaffolds-for-Delivery-of-Immunotherapies/ : 0.100309141967
1784 : http://www.the-scientist.com//?articles.view/articleNo/49003/title/Circadian-Rhythms-Influence-Treatment-Effects/ : 0.203546014688
1785 : http://www.the-scientist.com//?articles.view/articleNo/49008/title/Targeting-Tregs-Halts-Cancer-s-Immune-Helpers/ : 0.176295893878
1786 : http://www.the-scientist.com//?articles.view/articleNo/49006/title/Circadian-Clock-Affects-Health-and-Disease/ : 0.14616916
1787 : http://www.the-scientist.com//?articles.view/articleNo/49055/title/CRISPR-Screen-Detects-Functional-Gene-Regulation/ : 0.204584839319
1788 : http://www.the-scientist.com//?articles.view/articleNo/48906/title/Extra-Centrosomes-Can-Drive-Tumor-Formation-in-Mice/ : 0.0809558892734
1789 : http://www.the-scientist.com//?articles.view/articleNo/48863/title/Hitting-It-Out-of-the-Park/ : 0.173256138504
1790 : http://www.the-scientist.com//?articles.vie

1856 : https://www.scientificamerican.com/article/can-the-amazon-save-the-planet/ : 0.0236387241815
1857 : https://www.scientificamerican.com/article/national-corruption-breeds-personal-dishonesty/ : 0.0089299375
1858 : https://www.scientificamerican.com/article/the-world-wants-the-u-s-to-stay-in-the-paris-climate-deal/ : 0.00861119673469
1859 : https://www.scientificamerican.com/article/clean-power-worldwide-has-doubled-in-10-years/ : 0.00312222222222
1860 : http://www.the-scientist.com//?articles.view/articleNo/49066/title/Mutations-Linked-to-Secondary-Cancers/ : 0.166798686391
1861 : http://www.the-scientist.com//?articles.view/articleNo/49004/title/Infographic--Mechanisms-of-Resistance/ : 0.205244112222
1862 : http://neurosciencenews.com/patience-imagination-6340/ : 0.0
1863 : http://neurosciencenews.com/lsd-fear-emotion-6335/ : 0.0
1864 : http://neurosciencenews.com/hand-movement-pathway-6333/ : 0.0
1865 : http://neurosciencenews.com/depression-disability-health-6323/ : 0.0
1866 :

1936 : https://www.scientificamerican.com/article/the-arctic-ocean-is-becoming-more-like-the-atlantic-ocean/ : 0.00380398615917
1937 : https://www.scientificamerican.com/article/legendary-climate-scientist-likes-a-gop-proposal-on-global-warming/ : 0.0395888476331
1938 : https://www.scientificamerican.com/article/politicians-shouldnt-troll-through-scientists-e-mails/ : 0.002302484375
1939 : http://www.news-medical.net/news/20170410/Salk-Institute-receives-additional-243-million-Glenn-Foundation-gift-to-continue-aging-research.aspx : 0.00287565432099
1940 : http://www.news-medical.net/news/20170407/Patients-with-inducible-urticaria-can-benefit-from-treatment-with-asthma-drug.aspx : 0.005698609375
1941 : http://www.news-medical.net/news/20170411/TSRI-scientists-find-new-way-to-make-cell-population-resistant-to-HIV.aspx : 0.00701102469136
1942 : https://www.scientificamerican.com/article/a-last-ditch-attempt-to-save-the-world-rsquo-s-most-endangered-porpoise/ : 0.021038033241
1943 : http:/

2013 : http://neurosciencenews.com/artificial-intelligence-human-prejudice-6411/ : 0.0
2014 : http://neurosciencenews.com/immunotherapy-glioblastoma-6416/ : 0.0
2015 : http://neurosciencenews.com/alcohol-dependence-amygdala-6399/ : 0.0
2016 : http://neurosciencenews.com/artificial-intelligence-human-prejudice-6411/ : 0.0
2017 : http://neurosciencenews.com/oxytocin-ptsd-addiction-6412/ : 0.0
2018 : http://neurosciencenews.com/cognitive-map-scaling-6403/ : 0.0
2019 : http://neurosciencenews.com/sensory-cell-regeneration-hearing-6391/ : 0.0
2020 : http://neurosciencenews.com/oxytocin-ptsd-addiction-6412/ : 0.0
2021 : http://neurosciencenews.com/gambling-addiction-neuroimaging-6413/ : 0.0
2022 : http://neurosciencenews.com/plaand-genetics-6417/ : 0.0
2023 : https://www.washingtonpost.com/opinions/the-march-for-science-could-save-lives/2017/04/19/d042f956-2476-11e7-a1b3-faff0034e2de_story.html?utm_term=.62457cad7103 : 0.0
2024 : https://www.nytimes.com/2017/04/17/science/march-for-science-v

2100 : https://www.statnews.com/2017/04/25/oncology-cancer-precision-medicine-gleevec/ : 0.0277298302469
2101 : http://www.news-medical.net/news/20170421/Mice-study-yields-important-clues-about-protective-role-of-Clostridia-strains.aspx : 0.00555376559546
2102 : http://www.news-medical.net/news/20170419/Tips-to-help-prevent-allergies-during-spring-season.aspx : 0.0135517806122
2103 : http://www.news-medical.net/news/20170426/Unique-microscope-reveals-clues-to-destructive-autoimmune-disease.aspx : 0.00621790532544
2104 : http://www.news-medical.net/news/20170418/Cytokine-plays-critical-role-in-driving-immune-cells-to-promote-or-suppress-inflammatory-bowel-disease.aspx : 0.01037864
2105 : http://www.news-medical.net/news/20170424/UTSWc2a0researchers-develop-new-nanoparticle-vaccine-immunotherapy-that-targets-multiple-tumor-types.aspx : 0.0128298571429
2106 : http://www.news-medical.net/news/20170419/LIH-scientists-discover-unknown-immune-activation-mechanism-to-ward-off-pathogens.aspx : 

Let's create a checkpoint file to save our progress!

In [124]:
# news_df.to_json('data/news_checkpoint1.json')
# checkpoint_df = pd.read_json('data/news_checkpoint.json')
# checkpoint_df = checkpoint_df.sort_index(ascending=True)
# checkpoint_df = checkpoint_df[['articleUrl', 'categories', 'flagged', 'source', 'sourceUrl', 'title', 'article',
#                    'num_words_not_in_dictionary', 'stopwords2words', 'readability_score', 'sentiment_polarity']]
# news_df = checkpoint_df

# Explore TFIDF
### So far we have 3 important variables: news_df for all the news, articles for list/corpus of all articles, and vocab_df for the vocabularies of the articles
Now, I will explore TFIDF to extract keywords and try to match them to a categorizing model. This mapping can perhaps compare the results of the two algorithms. Many of the columns I populated above are not used, but they are interesting to look at and can be features in future models!<br><br>
Quoted directly from http://brandonrose.org/clustering#Stopwords,-stemming,-and-tokenizingHere.<br>
"I define term frequency-inverse document frequency (tf-idf) vectorizer parameters and then convert the synopses list into a tf-idf matrix. To get a Tf-idf matrix, first count word occurrences by document. This is transformed into a document-term matrix (dtm). This is also just called a term frequency matrix. An example of a dtm is here at right. Then apply the term frequency-inverse document frequency weighting: words that occur frequently within a document but not frequently within the corpus receive a higher weighting as these words are assumed to contain more meaning in relation to the document. A couple things to note about the parameters I define below: <br>-- max_df: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining (in the context of film synopses) <br>-- min_idf: this could be an integer (e.g. 5) and the term would have to be in at least 5 of the documents to be considered. Here I pass 0.2; the term must be in at least 20% of the document. I found that if I allowed a lower min_df I ended up basing clustering on names--for example "Michael" or "Tom" are names found in several of the movies and the synopses use these names frequently, but the names carry no real meaning. <br>-- ngram_range: this just means I'll look at unigrams, bigrams and trigrams."

In [261]:
# Define vectorizer parameters using Gridsearch (later in this notebook)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=0.01, max_df = 0.8, use_idf=True, ngram_range=(1,1),
                                   tokenizer=tokenize_and_stem)

tfidf_matrix = tfidf_vectorizer.fit_transform(articles) #fit the vectorizer to descriptions
terms = tfidf_vectorizer.get_feature_names() # terms is just a list of the features used in the tf-idf matrix.
print(tfidf_matrix.shape)
terms

(2150, 3235)


['abandon',
 'abil',
 'abl',
 'abnorm',
 'abolish',
 'abov',
 'abroad',
 'absenc',
 'absolut',
 'absorb',
 'abstract',
 'abund',
 'abus',
 'aca',
 'academ',
 'academi',
 'acceler',
 'accept',
 'access',
 'accid',
 'accommod',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accumul',
 'accur',
 'accuraci',
 'accus',
 'achiev',
 'acid',
 'acknowledg',
 'acquir',
 'acquisit',
 'act',
 'action',
 'activ',
 'activist',
 'actual',
 'acut',
 'ad',
 'adam',
 'adapt',
 'add',
 'addict',
 'addit',
 'address',
 'adequ',
 'adher',
 'adjust',
 'administ',
 'administr',
 'admit',
 'adolesc',
 'adopt',
 'adult',
 'advanc',
 'advantag',
 'advers',
 'advertis',
 'advic',
 'advis',
 'advisor',
 'advisori',
 'advoc',
 'advocaci',
 'affair',
 'affect',
 'affili',
 'afford',
 'afraid',
 'africa',
 'african',
 'afternoon',
 'afterward',
 'age',
 'agenc',
 'agenda',
 'agent',
 'aggreg',
 'aggress',
 'ago',
 'agre',
 'agreement',
 'agricultur',
 'ahead',
 'ai',
 'aid',
 'ailment',
 'aim',
 'air',
 'airp

### Clustering attempt to extract common keywords
Using the tf-idf matrix, I can now cluster the articles with k-means clustering. Since this a ML model that often converges to a "local optimum", sometimes these cells need to be run many times.

In [284]:
from sklearn.cluster import KMeans
num_clusters = 5 # played around with this number, and decided on 6 clusters / 6 main categories of the articles
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [288]:
from sklearn.externals import joblib
# joblib.dump(km, 'data/km.pkl') 
km = joblib.load('data/km.pkl') 

In [291]:
urls = [str(t) for t in news_df['articleUrl']]
categories = [c for c in news_df['categories']]# just to compare the cluster and the actual category
news_articles = { 'url': urls, 'article': articles, 'cluster': clusters, 'categories': categories}
cluster_df = pd.DataFrame(news_articles, index = [clusters] , columns = ['url', 'article', 'cluster', 'categories'])
print(cluster_df['cluster'].value_counts()) #number of films per cluster (clusters from 0 to 5)
cluster_df.head()

0    573
1    508
2    435
3    377
4    257
Name: cluster, dtype: int64


url  \
0  https://www.statnews.com/2017/02/09/dana-farbe...   
0  https://www.statnews.com/2017/02/09/antibiotic...   
2  https://www.statnews.com/2017/02/07/scientist-...   
0  https://www.statnews.com/2017/02/09/twitter-ho...   
0  https://www.statnews.com/2017/02/09/obamacare-...   

                                             article  cluster  \
0  \nT\nhe Dana-Farber Cancer Institute, which ha...        0   
0  \nY\nou’ve heard it many times before from you...        0   
2  \nI\n’ve recently been thinking about this: Th...        2   
0  \nC\nlinical medicine is rapidly changing in t...        0   
0  \nW\nASHINGTON — As Republicans confront the t...        0   

         categories  
0  [science policy]  
0  [science policy]  
2  [science policy]  
0  [science policy]  
0  [science policy]

In [292]:
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster", i, "top words:", end='')
    for ind in order_centroids[i, :10]: # 10 words per cluster
#         print(terms[ind].split(' '))
#         print(vocab_df.ix[terms[ind].split(' ')].values.tolist())
        print(vocab_df.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=', ')
    print() #add whitespace
    print() #add whitespace
    
    # USE STEM
    
#     print("Cluster", i, "titles:", end='')
#     for title in cluster_df.ix[i]['title'].values.tolist():
#         print(title, end='')
#     print() #add whitespace
#     print() #add whitespace

Cluster 0 top words:b'said', b'health', b'patients', b'care', b'trump', b'medical', b'climate', b'hospital', b'drugs', b'states', 

Cluster 1 top words:b'brain', b'research', b'studies', b'imaging', b'neurons', b'http', b'neurosciencenews', b'sleeping', b'active', b'february', 

Cluster 2 top words:b'ai', b'science', b'research', b'machine', b'data', b'use', b'learn', b'say', b'computers', b'like', 

Cluster 3 top words:b'plus', b'stat', b'subscribe', b'logging', b'biotechs', b'drugs', b'reads', b'story', b'article', b'available', 

Cluster 4 top words:b'cell', b'cancer', b'gene', b'immune', b'research', b'tumors', b'protein', b'patients', b'diseases', b'neurons', 



Looks like Cluster 0 represents science policy/environmental science, Cluster 1 represents neuroscience, Cluster 2 represents artificial intelligence, Cluster 3 sort of represents pharmaceuticals, and Cluster 4 represnts immunology. This clustering will be the first D3 visualization!

# Visualization 
"dist is defined as 1 - the cosine similarity of each document. Cosine similarity is measured against the tf-idf matrix and can be used to generate a measure of similarity between each document and the other documents in the corpus (each synopsis among the synopses). Subtracting it from 1 provides cosine distance which I will use for plotting on a euclidean (2-dimensional) plane.
Note that with dist it is possible to evaluate the similarity of any two or more synopses."

In [299]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
dist

array([[ -2.22044605e-16,   9.36636486e-01,   9.37527487e-01, ...,
          9.49005120e-01,   9.48263112e-01,   9.71655691e-01],
       [  9.36636486e-01,   0.00000000e+00,   8.74903476e-01, ...,
          9.16889500e-01,   9.39226185e-01,   9.49783936e-01],
       [  9.37527487e-01,   8.74903476e-01,   2.22044605e-16, ...,
          8.53410505e-01,   8.87167371e-01,   9.26460679e-01],
       ..., 
       [  9.49005120e-01,   9.16889500e-01,   8.53410505e-01, ...,
          1.11022302e-16,   8.55425870e-01,   8.89534648e-01],
       [  9.48263112e-01,   9.39226185e-01,   8.87167371e-01, ...,
          8.55425870e-01,  -4.44089210e-16,   9.12466078e-01],
       [  9.71655691e-01,   9.49783936e-01,   9.26460679e-01, ...,
          8.89534648e-01,   9.12466078e-01,   0.00000000e+00]])

In [300]:
from sklearn.manifold import MDS
MDS()
# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(dist)  # shape (n_components, n_samples)
xs, ys = pos[:, 0], pos[:, 1]
xs, ys

(array([-0.43890913, -0.15147367,  0.2856028 , ...,  0.47134885,
         0.58932929,  0.59853611]),
 array([-0.47929189, -0.16887924,  0.08910257, ...,  0.06806458,
         0.06912784,  0.27755768]))

In [326]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

#set up cluster names using a dict
cluster_names = {0: 'Sci Policy/Env Sci: trump, climate, states', 
                 1: 'Neuroscience: brain, imaging, neurons', 
                 2: 'Artificial Intelligence: ai, machine, data', 
                 3: 'Pharmaceuticals: biotechs, dugs', 
                 4: 'Immunology: cell, cancer, gene, immune, tumors, protein, parient, disease'}

#create data frame that has the result of the MDS plus the cluster numbers and titles
mds_df = pd.DataFrame(dict(x=xs, y=ys, cluster=clusters, categories=categories, url=urls)) 
mds_df.to_csv('data/mds_df.csv', index=False)
mds_df.head()

categories  cluster  \
0  [science policy]        0   
1  [science policy]        0   
2  [science policy]        2   
3  [science policy]        0   
4  [science policy]        0   

                                                 url         x         y  
0  https://www.statnews.com/2017/02/09/dana-farbe... -0.438909 -0.479292  
1  https://www.statnews.com/2017/02/09/antibiotic... -0.151474 -0.168879  
2  https://www.statnews.com/2017/02/07/scientist-...  0.285603  0.089103  
3  https://www.statnews.com/2017/02/09/twitter-ho... -0.396158 -0.409840  
4  https://www.statnews.com/2017/02/09/obamacare-...  0.317679 -0.304971

To wrap up the visualization, let's also generate a word cloud for all the vocab in vocab_df.

In [335]:
full_text = [w for w in vocab_df['words'] if w not in stopwords and len(w) >= 5]
fdist = nltk.FreqDist(full_text)

In [339]:
df = pd.DataFrame(fdist.most_common(50), columns=["word", "count"])
df.to_csv("data/word_cloud.csv", index=False)
df

word  count
0           research   4983
1              brain   4705
2              study   4108
3             health   3671
4              could   3464
5         university   3366
6             people   3359
7           patients   3278
8              would   3127
9              cells   3082
10       researchers   2960
11             first   2344
12         published   2045
13             human   2025
14             image   2000
15           medical   1996
16             found   1957
17           science   1950
18            cancer   1931
19             march   1855
20             years   1821
21           disease   1815
22          february   1739
23            system   1643
24           percent   1581
25         treatment   1498
26           neurons   1468
27        scientists   1418
28             using   1409
29            author   1382
30         different   1375
31            online   1357
32             might   1328
33         institute   1326
34             trump   1310
35           results   1298
36           studies   1277
37          children   1276
38          national   1247
39      neuroscience   1245
40          medicine   1225
41       information   1224
42  neurosciencenews   1172
43            change   1153
44            called   1153
45           whether   1137
46          activity   1132
47             group   1124
48         professor   1122
49             sleep   1119